In [1]:
from glob import glob
import xarray as xr
import cftime
import nc_time_axis
import numpy as np
import matplotlib.pyplot as plt
import intake, intake_esm

In [2]:
def timechunk_func(ds):
    """chunk time:1 for datasets that have time as dim
        Input: dataset 
        """
    if 'time' in ds.dims:
        ds = ds.chunk({'time':1})
    return ds

In [3]:
def rename_func(ds):
    """Rename dims"""
    if 'nlat' and 'nlon' in ds.dims:
        ds=ds.rename({'nlat':'y','nlon':'x'})
    return ds

In [19]:
def rename_and_chunk(ds):
    """rename dims and chunk time:1 for ds that has time as dimension"""
    if 'nlat' and 'nlon' in ds.dims:
        ds=ds.rename({'nlat':'y','nlon':'x'})
    if 'time' in ds.dims:
        ds = ds.chunk({'time':1})
    return ds

### Load the catalog 

In [4]:
col_url = "/nbhome/Mackenzie.Blanusa/gfdl-aws-analysis/esm-collection-spec-examples/intake-uda.json"

In [5]:
col = intake.open_esm_datastore(col_url)
col

,unique
project,1
institute,38
model,95
experiment_id,218
frequency,0
modeling_realm,0
mip_table,37
ensemble_member,2725
grid_label,14
variable,738


In [6]:
col.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
0,CMIP6,IPSL,IPSL-CM6A-LR,1pctCO2,NaN,NaN,Eday,r1i1p1f1,gr,ta850,18500101-18691231,v20180727,/uda/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/1pctCO2/r1i1...
1,CMIP6,IPSL,IPSL-CM6A-LR,1pctCO2,NaN,NaN,Eday,r1i1p1f1,gr,zg,18500101-18891231,v20180727,/uda/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/1pctCO2/r1i1...
2,CMIP6,IPSL,IPSL-CM6A-LR,1pctCO2,NaN,NaN,Eday,r1i1p1f1,gr,va,18500101-18891231,v20180727,/uda/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/1pctCO2/r1i1...
3,CMIP6,IPSL,IPSL-CM6A-LR,1pctCO2,NaN,NaN,Eday,r1i1p1f1,gr,hus,18500101-18891231,v20180727,/uda/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/1pctCO2/r1i1...
4,CMIP6,IPSL,IPSL-CM6A-LR,1pctCO2,NaN,NaN,Eday,r1i1p1f1,gr,ta500,18500101-18691231,v20180727,/uda/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/1pctCO2/r1i1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
992379,CMIP6,NCC,NorESM2-LM,esm-pi-cdr-pulse,NaN,NaN,Emon,r1i1p1f1,gn,zg,186001-186912,v20191108,/uda/CMIP6/CDRMIP/NCC/NorESM2-LM/esm-pi-cdr-pu...
992380,CMIP6,NCC,NorESM2-LM,esm-pi-cdr-pulse,NaN,NaN,Emon,r1i1p1f1,gn,zg,190001-190912,v20191108,/uda/CMIP6/CDRMIP/NCC/NorESM2-LM/esm-pi-cdr-pu...
992381,CMIP6,NCC,NorESM2-LM,esm-pi-cdr-pulse,NaN,NaN,Emon,r1i1p1f1,gn,zg,187001-187912,v20191108,/uda/CMIP6/CDRMIP/NCC/NorESM2-LM/esm-pi-cdr-pu...
992382,CMIP6,NCC,NorESM2-LM,esm-pi-cdr-pulse,NaN,NaN,Emon,r1i1p1f1,gn,zg,185001-185912,v20191108,/uda/CMIP6/CDRMIP/NCC/NorESM2-LM/esm-pi-cdr-pu...


In [7]:
cat = col.search(experiment_id=['historical'],
                 mip_table='Omon',
                 ensemble_member="r1i1p1f1",
                 model=['GFDL-CM4','CESM2'],
                 grid_label='gn',
                 variable=["thetao"])

cat1 = col.search(experiment_id=['historical'],
                 mip_table=['Omon','Ofx'],
                 ensemble_member="r1i1p1f1",
                 model=['GFDL-CM4','CESM2'],
                 grid_label='gn',
                 variable=["volcello"])

In [8]:
cat

,unique
project,1
institute,2
model,2
experiment_id,1
frequency,0
modeling_realm,0
mip_table,1
ensemble_member,1
grid_label,1
variable,1


In [9]:
cat1

,unique
project,1
institute,2
model,2
experiment_id,1
frequency,0
modeling_realm,0
mip_table,2
ensemble_member,1
grid_label,1
variable,1


In [10]:
cat.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
0,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,195001-196912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
1,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,187001-188912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
2,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,189001-190912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
3,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,197001-198912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
4,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,185001-186912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
5,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,191001-192912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
6,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,193001-194912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
7,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,201001-201412,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
8,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,199001-200912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
9,CMIP6,NCAR,CESM2,historical,NaN,NaN,Omon,r1i1p1f1,gn,thetao,185001-201412,v20190308,/uda/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1...


In [11]:
cat1.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
0,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,189001-190912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
1,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,187001-188912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
2,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,201001-201412,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
3,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,193001-194912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
4,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,197001-198912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
5,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,199001-200912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
6,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,195001-196912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
7,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,185001-186912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
8,CMIP6,NOAA-GFDL,GFDL-CM4,historical,NaN,NaN,Omon,r1i1p1f1,gn,volcello,191001-192912,v20180701,/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/...
9,CMIP6,NCAR,CESM2,historical,NaN,NaN,Ofx,r1i1p1f1,gn,volcello,NaN,v20190308,/uda/CMIP6/CMIP/NCAR/CESM2/historical/r1i1p1f1...


In [17]:
dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 1}},preprocess=rename_func)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [18]:
dset_dict

{'CMIP6.NCAR.CESM2.historical.Omon': <xarray.Dataset>
 Dimensions:          (d2: 2, ensemble_member: 1, lev: 60, time: 1980, vertices: 4, x: 320, y: 384)
 Coordinates:
     lat              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   * lev              (lev) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
     lon              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   * y                (y) int32 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
   * x                (x) int32 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
   * time             (time) object 1850-01-15 13:00:00.000007 ... 2014-12-15 ...
   * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
 Dimensions without coordinates: d2, vertices
 Data variables:
     thetao           (ensemble_member, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 60, 384, 320), meta=np.ndarray>
     time_bnds        (time, d2) object dask.array<chunksize=(1, 2), meta=np.ndarray>
     

In [32]:
dset_dict1 = cat1.to_dataset_dict(preprocess=rename_and_chunk)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


/nbhome/Raphael.Dussin/anaconda3/envs/mackenzie/lib/python3.8/site-packages/xarray/conventions.py:492: SerializationWarning: variable 'volcello' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [56]:
dset_dict1

{'CMIP6.NCAR.CESM2.historical.Ofx': <xarray.Dataset>
 Dimensions:          (d2: 2, ensemble_member: 1, lev: 60, vertices: 4, x: 320, y: 384)
 Coordinates:
     lat              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   * lev              (lev) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
     lon              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   * y                (y) int32 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
   * x                (x) int32 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
   * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
 Dimensions without coordinates: d2, vertices
 Data variables:
     volcello         (ensemble_member, lev, y, x) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
     lat_bnds         (y, x, vertices) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
     lon_bnds         (y, x, vertices) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndar

In [33]:
dset_dict1.keys(),dset_dict.keys()

(dict_keys(['CMIP6.NCAR.CESM2.historical.Ofx', 'CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']),
 dict_keys(['CMIP6.NCAR.CESM2.historical.Omon', 'CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']))

In [34]:
CM4_T = dset_dict['CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']['thetao'].squeeze(dim='ensemble_member')

In [35]:
CM4_V = dset_dict1['CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']['volcello'].squeeze(dim='ensemble_member')

In [36]:
CM4_T

<xarray.DataArray 'thetao' (time: 1980, lev: 35, y: 1080, x: 1440)>
dask.array<getitem, shape=(1980, 35, 1080, 1440), dtype=float32, chunksize=(1, 35, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) object 1950-01-16 12:00:00 ... 2009-12-16 12:00:00
  * x                (x) float64 -299.7 -299.5 -299.2 ... 59.53 59.78 60.03
  * y                (y) float64 -80.39 -80.31 -80.23 ... 89.73 89.84 89.95
  * lev              (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    lon              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    ensemble_member  <U8 'r1i1p1f1'
Attributes:
    long_name:      Sea Water Potential Temperature
    units:          degC
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello volume: volcello
    standard_name:  sea_water_potential_temperature
    original_name:  thetao

In [37]:
CM4_V

<xarray.DataArray 'volcello' (time: 1980, lev: 35, y: 1080, x: 1440)>
dask.array<getitem, shape=(1980, 35, 1080, 1440), dtype=float32, chunksize=(1, 35, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) object 1890-01-16 12:00:00 ... 1929-12-16 12:00:00
  * x                (x) float64 -299.7 -299.5 -299.2 ... 59.53 59.78 60.03
  * y                (y) float64 -80.39 -80.31 -80.23 ... 89.73 89.84 89.95
  * lev              (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    lon              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    ensemble_member  <U8 'r1i1p1f1'
Attributes:
    long_name:      Ocean Grid-Cell Volume
    units:          m3
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello volume: volcello
    standard_name:  ocean_volume
    original_name:  volcello

In [38]:
CESM2_T = dset_dict['CMIP6.NCAR.CESM2.historical.Omon']['thetao'].squeeze(dim='ensemble_member').mean(dim='time')

In [39]:
CESM2_T

<xarray.DataArray 'thetao' (lev: 60, y: 384, x: 320)>
dask.array<mean_agg-aggregate, shape=(60, 384, 320), dtype=float32, chunksize=(60, 384, 320), chunktype=numpy.ndarray>
Coordinates:
    lat              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * lev              (lev) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    lon              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * y                (y) int32 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
  * x                (x) int32 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
    ensemble_member  <U8 'r1i1p1f1'

In [42]:
CESM2_V = dset_dict1['CMIP6.NCAR.CESM2.historical.Ofx']['volcello'].squeeze(dim='ensemble_member')

In [43]:
CESM2_V

<xarray.DataArray 'volcello' (lev: 60, y: 384, x: 320)>
dask.array<getitem, shape=(60, 384, 320), dtype=float32, chunksize=(60, 384, 320), chunktype=numpy.ndarray>
Coordinates:
    lat              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * lev              (lev) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    lon              (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * y                (y) int32 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
  * x                (x) int32 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
    ensemble_member  <U8 'r1i1p1f1'
Attributes:
    cell_measures:  area: areacello volume: volcello
    cell_methods:   area: mean
    comment:        For oceans with more than 1 mesh (e.g. staggered grids), ...
    description:    For oceans with more than 1 mesh (e.g. staggered grids), ...
    frequency:      fx
    id:             volcello
    long_name:      Ocean Grid-Cell Volume
    mipTable:       Ofx
    out_name:       volcello
    prov:           fx ((isd.003))
    realm:          ocean
    standard_name:  ocean_volume
    time_label:     None
    time_title:     No temporal dimensions ... fixed field
    title:          Ocean Grid-Cell Volume
    type:           real
    units:          m3
    variable_id:    volcello

### Compute OHC

In [44]:
dset_dict.keys(),dset_dict1.keys()

(dict_keys(['CMIP6.NCAR.CESM2.historical.Omon', 'CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']),
 dict_keys(['CMIP6.NCAR.CESM2.historical.Ofx', 'CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']))

In [45]:
OHC = {}
for i,j in zip(dset_dict,dset_dict1):
        OHC[i]= 3992 * 1025 * dset_dict[i]['thetao'].squeeze(dim='ensemble_member') * dset_dict1[j]['volcello'].squeeze(dim='ensemble_member')

In [46]:
OHC.keys()

dict_keys(['CMIP6.NCAR.CESM2.historical.Omon', 'CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon'])

In [47]:
OHC

{'CMIP6.NCAR.CESM2.historical.Omon': <xarray.DataArray (time: 1980, lev: 60, y: 384, x: 320)>
 dask.array<mul, shape=(1980, 60, 384, 320), dtype=float64, chunksize=(1, 60, 384, 320), chunktype=numpy.ndarray>
 Coordinates:
     lat              (y, x) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
   * lev              (lev) float64 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
     lon              (y, x) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
   * y                (y) int32 1 2 3 4 5 6 7 8 ... 378 379 380 381 382 383 384
   * x                (x) int32 1 2 3 4 5 6 7 8 ... 314 315 316 317 318 319 320
   * time             (time) object 1850-01-15 13:00:00.000007 ... 2014-12-15 ...
     ensemble_member  <U8 'r1i1p1f1',
 'CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon': <xarray.DataArray (time: 1980, lev: 35, y: 1080, x: 1440)>
 dask.array<mul, shape=(1980, 35, 1080, 1440), dtype=float64, chunksize=(1, 35, 1080, 1440), chunktype=numpy.ndarray>
 Coordinates:
   * time             (t

In [53]:
global_OHC_upper700m = {}
for i in OHC:    
    global_OHC_upper700m[i] = OHC[i].sel(lev=slice(0,700)).sum(dim=('lev', 'y', 'x')).load()

In [63]:
#issues running this on dask
OHC['CMIP6.NCAR.CESM2.historical.Omon'].sel(lev=slice(0,700)).sum(dim=('lev', 'y', 'x')).load()

KeyboardInterrupt: 

In [59]:
global_OHC_upper700m

{'CMIP6.NCAR.CESM2.historical.Omon': <xarray.DataArray (time: 1980)>
 array([2.71942001e+23, 2.73162421e+23, 2.73192270e+23, ...,
        2.74690785e+23, 2.74237227e+23, 2.76415160e+23])
 Coordinates:
   * time             (time) object 1850-01-15 13:00:00.000007 ... 2014-12-15 ...
     ensemble_member  <U8 'r1i1p1f1',
 'CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon': <xarray.DataArray (time: 1980)>
 dask.array<sum-aggregate, shape=(1980,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
 Coordinates:
   * time             (time) object 1950-01-16 12:00:00 ... 2009-12-16 12:00:00
     ensemble_member  <U8 'r1i1p1f1'}

In [58]:
for i in global_OHC_upper700m: 
    %time _ = global_OHC_upper700m[i].load()

CPU times: user 1e+03 µs, sys: 0 ns, total: 1e+03 µs
Wall time: 159 µs


KilledWorker: ("('open_dataset-3ee0074259a92257f11c3c282ede88ddvolcello-6577e003828dddac0298f10d4887630d', 0, 0, 0, 0)", <Worker 'tcp://140.208.147.29:37793', name: 0-5, memory: 0, processing: 45>)